<a href="https://colab.research.google.com/github/viveksahukar/sih/blob/main/dicom_to_np_array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook details how the numpy array are extracted from Dicom files to be fed directly to CNN.

In [ ]:
# Import required libraries
from vs165_modules import *

## Reading axial slices first...

In [ ]:
# Reading the csv file
# use r string in file path since space in file folder name
ax = pd.read_csv(r"/storage/vs165-projects/Pro00103719 - SIH deep learning/data_ax_cor/data_ax/ax_t1spgr+c_series_master_copy.csv")
ax['label'] = np.where(ax['dirpath'].str.contains('Positive', case=False, regex=True), 1, 0)

ax.rename(columns={'dirpath':'fpath'}, inplace=True)
ax.info()

In [ ]:
ax_target_slice = partial(target_slice, targ_pct=0.7)
ax['targ_ax'] = ax['fpath'].map(ax_target_slice)
ax['full_fpath'] = ax['fpath'] + '/' + ax['targ_ax']
ax.drop(columns=['fpath', 'targ_ax'], inplace=True)

In [ ]:
# Saving ax_new to a dataframe and then csv, so all steps till here need not be repeated
ax.to_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/ax.csv', index=False)

## Now reading coronal slices - similar method as for axial slices....

In [ ]:
# Reading the csv file
# use r string in file path since space in file folder name
cor = pd.read_csv(r"/storage/vs165-projects/Pro00103719 - SIH deep learning/data_ax_cor/data_cor/cor_t1spgr+c_series_master_copy.csv")
cor['label'] = np.where(cor['dirpath'].str.contains('Positive', case=False, regex=True), 1, 0)

cor.rename(columns={'dirpath':'fpath'}, inplace=True)
cor.info()

In [ ]:
cor_1 = cor.copy() # Making copy of the coronal slices dataframe

In [ ]:
# Extracting coronal slice 1
cor_1_target_slice = partial(target_slice, targ_pct=0.3)
cor_1['targ_ax'] = cor_1['fpath'].map(cor_1_target_slice)
cor_1['full_fpath'] = cor_1['fpath'] + '/' + cor_1['targ_ax']
cor_1.drop(columns=['fpath', 'targ_ax'], inplace=True)

In [ ]:
# Saving ax_new to a dataframe and then csv, so all steps till here need not be repeated
cor_1.to_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/cor_1.csv', index=False)

In [ ]:
cor_1.info()

In [ ]:
cor_2 = cor.copy() # Making another copy of coronal slices dataframe

In [ ]:
# Extracting coronal slice 2
cor_2_target_slice = partial(target_slice, targ_pct=0.625)
cor_2['targ_ax'] = cor_2['fpath'].map(cor_2_target_slice)
cor_2['full_fpath'] = cor_2['fpath'] + '/' + cor_2['targ_ax']
cor_2.drop(columns=['fpath', 'targ_ax'], inplace=True)

In [ ]:
cor_2.info()

In [ ]:
# Saving ax_new to a dataframe and then csv, so all steps till here need not be repeated
cor_2.to_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/cor_2.csv', index=False)

## Getting common patients having both axial and coronal slices - limiting analysis to them only

In [ ]:
#Load csv dataframe
df_ax = pd.read_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/ax.csv')

#Load csv dataframe for axial images
df_cor_1 = pd.read_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/cor_1.csv')

#Load csv dataframe for axial images
df_cor_2 = pd.read_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/cor_2.csv')

## Merging axial and coronal patients having common Patient ID

In [ ]:
# Keep important columns only
ax = df_ax[['PatientID', 'SeriesDescription', 'label', 'full_fpath']]
cor_1 = df_cor_1[['PatientID', 'SeriesDescription', 'label', 'full_fpath']]
cor_2 = df_cor_2[['PatientID', 'SeriesDescription', 'label', 'full_fpath']]

# Get the common patients id in axial and coronal
ax_cor_1 = ax.merge(cor_1, how='inner', indicator='True', on='PatientID')

ax_cor_1 = ax_cor_1[['PatientID']] #extract only common PatientID

# Now merge this with both ax and cor to get common Patients only
ax_common = ax.merge(ax_cor_1, how='right', on='PatientID')

cor_common_1  = cor_1.merge(ax_cor_1, how='right', on='PatientID')
cor_common_2  = cor_2.merge(ax_cor_1, how='right', on='PatientID')

In [ ]:
# Now write these dataframes to csv, so steps don't need to be repeated and the csv files will be fed to dataloader
ax_common.to_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/ax_common.csv', index=False)
cor_common_1.to_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/cor_common_1.csv', index=False)
cor_common_2.to_csv(r'/storage/vs165-projects/Pro00103719 - SIH deep learning/test/cor_common_2.csv', index=False)

#### **Number of patients having slices for both axes - axial and coronal in positive and negative cases**
| Case | Both Axial & Coronal | 
| :-- | --: | 
| Positive | 155 | 
| Negative | 273 | 
| Total | 428 | 